In [14]:
import pandas as pd
import pickle
import os
import osmnx as ox
import folium
import pydeck as pdk
import networkx as nx
from tqdm import tqdm
import copy
import random
import numpy as np

In [ ]:
graph_path = './moscow.graphml'
print("Шаг 1: Загрузка через чистый networkx...")
G_raw = nx.read_graphml(graph_path)


G = nx.MultiDiGraph(G_raw)

for u, data in G.nodes(data=True):
    if 'x' in data and isinstance(data['x'], str):
        data['x'] = float(data['x'])
    if 'y' in data and isinstance(data['y'], str):
        data['y'] = float(data['y'])
    if 'osmid' in data and isinstance(data['osmid'], str):
        pass


Шаг 1: Загрузка через чистый networkx...


In [18]:
class PercolationSimulator:
    def __init__(self, filepath):
        """
        Загрузка графа из файла GraphML.
        """
        print(f"Загрузка графа из {filepath}...")
        # Загружаем как MultiDiGraph (ориентированный граф с возможными дубликатами ребер)

        G_raw = nx.read_graphml(graph_path)
        G = nx.MultiDiGraph(G_raw)

        for u, data in G.nodes(data=True):
            if 'x' in data and isinstance(data['x'], str):
                data['x'] = float(data['x'])
            if 'y' in data and isinstance(data['y'], str):
                data['y'] = float(data['y'])
            if 'osmid' in data and isinstance(data['osmid'], str):
               pass

        self.original_graph = G

        # Для упрощения расчетов перколяции часто граф преобразуют в неориентированный,
        # чтобы проверять физическую связность (есть ли вообще дорога между А и Б).
        # Но чтобы сохранить веса, используем Graph (не MultiGraph)
        self.working_graph = self.original_graph.to_undirected()

        print(f"Граф загружен: {len(self.working_graph.nodes)} узлов, {len(self.working_graph.edges)} ребер.")

    def calculate_lcc_fraction(self, graph):
        """
        Считает долю узлов в крупнейшей связной компоненте (LCC).
        Это наша главная метрика Y на графике.
        """
        if len(graph.nodes) == 0:
            return 0

        # Получаем список всех компонент связности
        components = list(nx.connected_components(graph))

        # Находим самую большую
        largest_component = max(components, key=len)

        # Считаем долю
        return len(largest_component) / len(self.working_graph.nodes)

    def get_edges_ranking(self, strategy='random'):
        """
        Возвращает список ребер, отсортированный по порядку удаления.
        strategy: 'random', 'betweenness' (функциональная важность), 'bridges' (структурная)
        """
        edges = list(self.working_graph.edges(keys=True)) # keys=True нужен для osmnx графов

        if strategy == 'random':
            random.shuffle(edges)
            return edges

        elif strategy == 'betweenness':
            print("Расчет Edge Betweenness Centrality (это может занять время)...")
            # ВНИМАНИЕ: Для больших графов (>2000 узлов) это будет ОЧЕНЬ долго.
            # Можно использовать k=... для приближенного расчета, например k=100
            centrality = nx.edge_betweenness_centrality(self.working_graph, weight='length') # или normalized=True

            # Сортируем ребра: от самых важных (высокий скор) к неважным
            sorted_edges = sorted(centrality.items(), key=lambda item: item[1], reverse=True)
            return [edge[0] for edge in sorted_edges]

        elif strategy == 'degree':
             # Удаляем ребра, инцидентные самым загруженным узлам (простая эвристика)
             # Реализация упрощенная, лучше использовать betweenness
             pass

        return edges

    def simulate(self, strategy='random', steps=50):
        """
        Основной цикл симуляции.
        steps: на сколько шагов разбить удаление (точек на графике).
        """
        # Делаем копию графа, чтобы не ломать основной
        G_temp = self.working_graph.copy()

        # Получаем список смертников (ребра в порядке удаления)
        edges_to_remove = self.get_edges_ranking(strategy)
        total_edges = len(edges_to_remove)

        # Результаты: [доля_удаленных, доля_LCC]
        results = {'removed_pct': [], 'lcc_pct': []}

        # Определяем, сколько ребер удалять за один шаг
        chunk_size = max(1, total_edges // steps)

        print(f"Запуск симуляции: {strategy}")

        # Сначала точка 0 (ничего не удалено)
        results['removed_pct'].append(0)
        results['lcc_pct'].append(self.calculate_lcc_fraction(G_temp))

        removed_count = 0

        # Идем по списку и удаляем пачками
        for i in tqdm(range(0, total_edges, chunk_size)):
            chunk = edges_to_remove[i : i + chunk_size]

            # Удаляем ребра
            G_temp.remove_edges_from(chunk)
            removed_count += len(chunk)

            # Записываем метрики
            fraction_removed = removed_count / total_edges
            lcc = self.calculate_lcc_fraction(G_temp)

            results['removed_pct'].append(fraction_removed)
            results['lcc_pct'].append(lcc)

            # Оптимизация: если сеть уже развалилась почти полностью, можно стопать
            if lcc < 0.05:
                break

        return results


In [ ]:
# 1. Инициализация (путь к файлу от Человека 1)
# Предположим, файл лежит тут:
sim = PercolationSimulator("data/moscow_test.graphml")

# 2. Запуск сценариев
# Случайное удаление (Random Failure)
res_random = sim.simulate(strategy='random')

# Целенаправленная атака (Targeted Attack)
res_targeted = sim.simulate(strategy='betweenness')

# 3. Сохранение результатов для Человека 3 (который строит графики)
df_rand = pd.DataFrame(res_random)
df_rand['strategy'] = 'Random'

df_target = pd.DataFrame(res_targeted)
df_target['strategy'] = 'Betweenness'

final_df = pd.concat([df_rand, df_target])
final_df.to_csv("./percolation_results.csv", index=False)
print("Готово! Данные сохранены в CSV.")

Загрузка графа из data/moscow_test.graphml...
Граф загружен: 2 узлов, 1 ребер.
Запуск симуляции: random


100%|██████████| 1/1 [00:00<00:00, 8081.51it/s]


Расчет Edge Betweenness Centrality (это может занять время)...
Запуск симуляции: betweenness


100%|██████████| 1/1 [00:00<00:00, 7898.88it/s]

Готово! Данные сохранены в CSV.
